In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
terminología= "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labourlawterminologyv1.nt"
eurovoc="/content/drive/My Drive/Seguimiento-Lucía/papers/ijhac/data/eurovoc-skos-ap-eu.rdf"

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

# Nombres esperados de los archivos
terminos_file = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_literales.txt"
eurovoc_file = "/content/drive/My Drive/Seguimiento-Lucía/papers/ijhac/data/eurovoc-skos-ap-eu.rdf"  # Archivo XML de EuroVoc

# Leer lista de términos
with open(terminos_file, 'r', encoding='utf-8') as f:
    terminos = [line.strip().lower() for line in f if line.strip()]

print(f"Total de términos leídos desde el archivo: {len(terminos)}")
print("Primeros 5 términos leídos:", terminos[:5])

# Parsear archivo XML
tree = ET.parse(eurovoc_file)
root = tree.getroot()

# Definir namespaces
NS_SKOS = '{http://www.w3.org/2004/02/skos/core#}'
NS_SKOSXL = '{http://www.w3.org/2008/05/skos-xl#}'

# Buscar definiciones
resultados = []

for concept in root.findall('.//rdf:Description', namespaces={'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'}):
    # Buscar etiquetas prefLabel (SKOS)
    etiquetas = concept.findall(f'{NS_SKOS}prefLabel')

    # Extraer texto de las etiquetas
    texto_etiquetas = [e.text.strip().lower() for e in etiquetas if e.text]

    # Buscar definiciones (SKOS)
    definiciones = concept.findall(f'{NS_SKOS}definition')

    # Extraer definición en español, o inglés si no hay en español
    definicion = None
    for lang in ['es', 'en']:
        definicion = next((d.text.strip() for d in definiciones
                          if d.attrib.get('{http://www.w3.org/XML/1998/namespace}lang') == lang), None)
        if definicion:
            break

    # Agregar coincidencias
    for etiqueta in texto_etiquetas:
        if etiqueta in terminos:
            resultados.append({'término': etiqueta, 'definición': definicion})

# Ver resultados
print(f"Total de resultados encontrados: {len(resultados)}")
if resultados:
    print("Primeros 5 resultados:", resultados[:5])

# Guardar a CSV
df = pd.DataFrame(resultados)
csv_file = '/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_con_definiciones.csv'
df.to_csv(csv_file, index=False, encoding='utf-8')

print(f"El archivo CSV ha sido guardado exitosamente en: {csv_file}")



Total de términos leídos desde el archivo: 187
Primeros 5 términos leídos: ['trabajo nocturno', 'seguridad de los alimentos', 'sistema de alerta de la ue', 'maternidad', 'persona jurídica']
Total de resultados encontrados: 205
Primeros 5 resultados: [{'término': 'organización del trabajo', 'definición': None}, {'término': 'restricción de la competencia', 'definición': None}, {'término': 'política de la vivienda', 'definición': None}, {'término': 'vivienda', 'definición': None}, {'término': 'concentración económica', 'definición': None}]
El archivo CSV ha sido guardado exitosamente en: /content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_con_definiciones.csv


In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
import csv
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import SKOS
import unicodedata

# Cargar grafo RDF
g = Graph()
g.parse(eurovoc, format="xml")  # <-- Cambia aquí tu archivo RDF

# Leer términos desde el TXT
with open("/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_literales.txt", "r", encoding="utf-8") as f:
    terminos = [line.strip() for line in f if line.strip()]

# Normalizar texto
def normalizar(texto):
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    return texto.lower()

# Namespaces
SKOSXL = Namespace("http://www.w3.org/2008/05/skos-xl#")

# Funciones auxiliares
def obtener_pref_label(uri):
    for _, _, label in g.triples((uri, SKOSXL.prefLabel, None)):
        for _, _, literal in g.triples((label, SKOSXL.literalForm, None)):
            if literal.language == "es":
                return str(literal)
    for _, _, label in g.triples((uri, SKOS.prefLabel, None)):
        if label.language == "es":
            return str(label)
    return None

def obtener_dominios(uri):
    dominios = []
    for _, _, dominio_uri in g.triples((uri, SKOS.inScheme, None)):
        etiqueta = obtener_pref_label(dominio_uri)
        if etiqueta and etiqueta.lower() != "eurovoc":
            dominios.append(etiqueta)
    return dominios

def obtener_relaciones(uri, relacion):
    relacionados = []
    for _, _, relacionado_uri in g.triples((uri, relacion, None)):
        etiqueta = obtener_pref_label(relacionado_uri)
        if etiqueta:
            relacionados.append(etiqueta)
    return relacionados

def obtener_definicion(uri):
    # Extraemos la definición en español si está disponible
    for _, _, definition in g.triples((uri, SKOS.definition, None)):
        for _, _, literal in g.triples((definition, SKOSXL.literalForm, None)):
            if literal.language == "es":
                return str(literal)
    return None

def obtener_alternative_labels(uri):
    alternative_labels = []
    for _, _, alt_label in g.triples((uri, SKOSXL.altLabel, None)):
        for _, _, literal in g.triples((alt_label, SKOSXL.literalForm, None)):
            if literal.language == "es":
                alternative_labels.append(str(literal))
    return alternative_labels

# Crear índice de conceptos
conceptos_idx = {}
for s in g.subjects(predicate=SKOS.prefLabel):
    label = obtener_pref_label(s)
    if label:
        conceptos_idx[normalizar(label)] = s

# Preparar filas para el CSV
filas = []

for termino in terminos:
    termino_norm = normalizar(termino)
    uri = conceptos_idx.get(termino_norm)

    broader = []
    narrower = []
    dominios = []
    alternative_labels = []
    definicion = None

    if uri:
        definicion = obtener_definicion(uri)
        broader = obtener_relaciones(uri, SKOS.broader)
        narrower = obtener_relaciones(uri, SKOS.narrower)
        dominios = obtener_dominios(uri)
        alternative_labels = obtener_alternative_labels(uri)

    # Crear contexto basado en la disponibilidad de la definición
    if definicion:
        contexto_str = definicion
    else:
        partes_contexto = []

        if dominios:
            partes_contexto.append("; ".join(dominios))
        if alternative_labels:
            partes_contexto.append("; ".join(alternative_labels))
        if broader:
            partes_contexto.append("; ".join(broader))
        if narrower:
            partes_contexto.append("; ".join(narrower))

        # Eliminar números y asegurarse de que el contexto no contenga "Eurovoc"
        contexto_str = " | ".join(partes_contexto)
        contexto_str = ''.join(c for c in contexto_str if not c.isdigit())

    filas.append([termino, contexto_str])

# Escribir CSV
with open('/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/eurovoc_con_definiciones.csv', "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    writer.writerow(["Término", "Contexto"])
    writer.writerows(filas)

print("\n✅ CSV 'resultados_contexto_definiciones.csv' creado correctamente.")



✅ CSV 'resultados_contexto_definiciones.csv' creado correctamente.
